## 1. Introduction: Getting started with MuSpAn

In this section we will:
- Create a MuSpAn Domain
- Add spatial data (with annotations) to this domain from a csv file 
- Visualise our data
- Generate the boundary of our data
- Explore the query methods in MuSpAn

### Tips:
This section will cover the basics in our tutorial series 'Getting Started' and 'Queries' which can be found at https://docs.muspan.co.uk/latest/tutorials.html. 




### Installing MuSpAn (if not done already)

If you haven't already, we can install MuSpAn like any other Python library using the pip package manager. See https://docs.muspan.co.uk/latest/Installation.html for more details.

Once you have followed the necessary steps, you should have MuSpAn ready for spatial analysis. Let's check this.

In [ ]:
import muspan as ms
print('MuSpAn was successfully imported!')

---

### An introductory walkthrough analysis 

In order to access the spatial analysis tools provided by MuSpAn, it needs to know what your data looks like. In this tutorial, we introduce the concept of a <i>domain</i>, the central object of interest within MuSpAn. You can think of the domain as being a container holding all the information relevant to a single spatial frame of reference, such as a region of interest. Let's make a new domain now, by calling ```ms.domain()```:

In [ ]:
# Create a new domain with the name 'Example'
my_domain = ms.domain('Example')

#print out the domain
print(my_domain)

This domain is pretty boring at the moment. ```ms.domain()``` requires us to give the domain a name (useful for when we're working with multiple regions of interest at once), here 'Example'. Calling ```print(my_domain)``` tells us about the contents of the domain, but at the moment there's nothing there. We need to populate our domain with some data.

Spatial data is stored in MuSpAn as <i>objects</i>: each object can be a <i>point</i>, <i>line</i>, or <i>shape</i>. These objects can represent anything we'd like. For example, we could have points representing cells, transcripts, or, shapes representing cellular boundaries, tissue regions of interest annotated in other software such as QuPath. It all depends on our interpretation of that object - the software doesn't care!

Domains are not static objects, they update as we interact with them through our analysis. For example, we can add and delete objects. Labels can be added from predefined annotations or generated through a specific spatial method. That is to say, our domain is an editable *bucket* of spatial data.

To see this in action, let's get a spatial dataset of a sample of murine colorectal carcinoma stored in the ```muspan.datasets``` submodule. This dataset is stored in a csv file which is a typical format for this type of data following image segmentation and annotation. The libraries ```pandas``` and ```os``` useful to handle data and file management - we'll import these to load in our csv file.

This sample was preprocessed using Halo. Cells were segmented and labelled using range of immune and epithelial markers. For more information on this dataset, see https://doi.org/10.1017/S2633903X24000011.

In [ ]:
import pandas as pd
import os

# Load the dataset
# The dataset is stored within MuSpAn for use as an example
# We read the CSV file into a pandas DataFrame
df = pd.read_csv(os.path.dirname(ms.datasets.__file__) + '/data/Bull_2024_mouse_colon.csv')

# Display the first 5 rows of the dataframe to understand its structure
print(df)

In this dataset, we have:
1. segmented cells represented by their centroid which are define by their (x,y) coordinates,
2. a phenotypic classification given in the 'Cell type' column,
3. the mean channel intensity of the CD4 marker for all cells.


To start, we'll just take a focus on the loading in the cells as points.

In [ ]:
# Import numpy library
import numpy as np

# Extract 'x' and 'y' columns from the dataframe and convert them to a numpy array
points = np.asarray([df['x'], df['y']])

# Print the shape of the resulting numpy array
print(np.shape(points))

Note that stacking the x and y columns together has generated a (2 x 6676) numpy array. In order to add this to a MuSpAn domain, we need it to be (n x 2) dimensional, so we’ll have to transpose the array to add the points. Let’s make a MuSpAn domain, add the (transposed) array of points.

In [19]:
# Add the transposed array of points to the domain with the label 'Cell centres'
my_domain.add_points(points.T, 'Cell centres')

Let's take a look to make sure our cells are correctly loaded in. We can visualise our domain using the ```muspan.visualise``` module. MuSpAn isn't intended to be a visualisation tool, but it's always handy to be able to see our data. MuSpAn uses [matplotlib](https://matplotlib.org/) to provide some handy tools to see your data quickly, so we'll import that too. Let's take a look at our points, using the ```ms.visualise.visualise()``` command:

In [ ]:
# Import matplotlib for plotting
import matplotlib.pyplot as plt


# Create a figure for visualisation
plt.figure(figsize=(8, 5))

# Visualise the domain
ms.visualise.visualise(my_domain, ax=plt.gca())

Metadata can be linked with MuSpAn objects using <i>labels</i>. In this case, let's add a label to each cell centre describing its cell type. Adding labels is a straightforward process, we just call ```domain.add_labels('Celltype',celltype_array)```, to add an array of labels with the name "Celltype" to the domain.

In [ ]:
# Extract the 'Cell type' column from the dataframe
celltype_array = df['Cell type']

# Add the cell type labels to the domain
my_domain.add_labels('Celltype', celltype_array)

# Print the domain to verify the labels have been added
print(my_domain)

We can see that the list of available Labels now contains 'Celltype'.

We can even view our labels with the `print_labels()` method of domain.

In [ ]:
my_domain.print_labels()

If we want to see what these labels look like, we can add an argument to the command ```ms.visualise.visualise(domain, 'Celltype')```, which MuSpAn will interpret as wanting to colour the objects in ```domain``` using a label called ```celltype```:

In [ ]:
# Visualise the domain, colouring objects by the 'Celltype' label
ms.visualise.visualise(my_domain, 'Celltype')

By default, MuSpAn will try to select a sensible colour palette for visualisation, but there are many ways to customise visualisation in MuSpAn (see the tutorials on visualisation for examples). For now, let's manually change some of the colors associated with this label to something with a bit less orange in it. We can do that by passing a dictionary of the colours we'd like to update to ```domain.update_colors```. Let's make the epithelium grey, and use the orange colour for T Helper Cells. MuSpAn uses ```pyplot``` for plotting, so we can specify colours in any way that ```matplotlib``` recognises. While we're here, we'll also use slightly smaller markers so that nearby points don't overlap.

In [ ]:
# Define new colors for specific cell types
new_colors = {'Epithelium': 'grey', 'T Helper Cell': plt.cm.tab10(1)}

# Update the colors in the domain for the 'Celltype' label
my_domain.update_colors(new_colors, label_name='Celltype')

# Create a new figure for visualisation
plt.figure(figsize=(8, 5))

# Visualise the domain, colouring objects by the 'Celltype' label with updated colors and smaller markers
ms.visualise.visualise(my_domain, 'Celltype', ax=plt.gca(), marker_size=5)

We may also want to show just certain cell types, for instance, only the Regulatory T Cells. We can do that by passing a tuple ```(label_name,label_value)``` as the keyword argument ```objects_to_plot```.

In [ ]:
# Create a new figure for visualisation
plt.figure(figsize=(8, 5))

# Visualise the domain, showing only the 'Regulatory T Cell' type
# The 'show_boundary' argument is set to True to display the boundary of the domain
ms.visualise.visualise(my_domain, 'Celltype', objects_to_plot=('Celltype', 'Regulatory T Cell'), ax=plt.gca(), show_boundary=True)

'Celltype' here is an example of a <i>categorical</i> label, in which the label takes distinct values. We can also assign <i>continuous</i> labels, such as the 'CD4 intensity' label included with this dataset (which describes the average expression of the marker CD4, found on T Helper Cells).

In [ ]:
# Extract the 'CD4 intensity' column from the dataframe
CD4_intensity = df['CD4 intensity']

# Add the CD4 intensity labels to the domain with a colormap 'viridis'
my_domain.add_labels('CD4', CD4_intensity, cmap='viridis')

# Create a new figure for visualisation
plt.figure(figsize=(8, 5))

# Visualise the domain, colouring objects by the 'CD4' label with smaller markers
ms.visualise.visualise(my_domain, 'CD4', ax=plt.gca(), marker_size=5)

A crucial part of the domain is the <i>boundary</i> - in order to do spatial analysis, we need to know not just what objects we're analysing, but also what space they live in! Just because we haven't specified a boundary doesn't mean that it isn't there. MuSpAn automatically infers a domain boundary whenever we make a domain, or add new data. By default, this boundary will be the minimal axis aligned bounding box for the data. The boundary is a special type of <i>shape</i> object - it can be accessed easily via ```my_domain.boundary```, e.g.:

In [ ]:
print(my_domain.boundary)

he boundary always has ID -1, and in this case we can see that it is formed from 4 *vertices*. Every object in MuSpAn is made of vertices. These are just 2D coordinates that define the object. Every point is made of 1 vertex, a line is an ordered of vertices, and a shape is an ordered list of vertices that finishes where it started (note that MuSpAn has other requirements for shapes and lines, such as not being self-intersecting, but let's not worry about that for now).

Our boundary shape has 4 vertices - where are they? By default, ```ms.visualise.visualise()``` doesn't show the domain boundary, but we can easily turn it on by passing the argument ```show_boundary=True```.

In [ ]:
# Create a new figure with a specified size
plt.figure(figsize=(8, 5))

# Visualise the domain using MuSpAn's visualisation tool
# This will plot the points we added to the domain and show the boundary
ms.visualise.visualise(my_domain,color_by='Celltype', ax=plt.gca(), show_boundary=True,marker_size=5)

# Print the area of the domain boundary
print(f'Domain area: {my_domain.boundary.area}')

---

### Queries in MuSpAn

One of the most powerful aspects of MuSpAn is its ```query``` module. This allows the user the user to select specific objects within the domain according to some desired criteria, extract those objects, and perform actions on them (such as detailed spatial analyses, plotting, and so on). This is particularly useful when a domain contains many different objects, but we want to focus our analysis on some subset of them.


You've already seen how tuples, for example, `(label_name,label)` can be passed to generate a subset of the data to visualise. But this is infact one of the many ways we can query data in our domain.

Let's look a bit closer at the `('Celltype', 'Regulatory T Cell')` command we used above.

Behind the scenes, a little bit of magic has just taken place. The ```objects_to_plot``` argument of ```visualise``` is really looking for a list of object IDs, which will tell ```visualise``` which ones to draw. Instead of passing it a list of integers, we've instead passed a tuple, ```('Celltype', 'Regulatory T Cell')```, that MuSpAn has converted into a list of IDs behind the scenes. This tuple is "query-like", MuSpAn terminology for an argument that can be interpreted by the domain as a list of object IDs. This particular tuple is interpreted by MuSpAn as a request for any objects which match the criteria "Object has a label in 'Celltype' and it is 'Regulatory T Cell'".

This type of tuple can also be used to isolate, for example, objects with a certain label. Let's plot only objects with the "Celltype" label equal to "Epithelium".



In [ ]:
ms.visualise.visualise(my_domain, 'Celltype', objects_to_plot=('Celltype', 'Epithelium'), show_boundary=True)

In the last tutorial, we saw how MuSpAn can interpret certain tuples as specifying sets of objects. In this tutorial, we'll explore the underlying ```muspan.query``` module in detail, and explain which tuples MuSpAn can convert into a query. We'll go into quite a lot of detail regarding what's happening 'under the hood', so it's not necessary to understand everything in this tutorial, and it can be skipped if desired.

MuSpAn queries ultimately rely on one particular class, ```muspan.query.query```, which is designed to standardise the format of a [Boolean query](https://en.wikipedia.org/wiki/Boolean_algebra). Each query has three parts: a `property`, a `relation`, and a `comparison_value` (query also takes the domain in as an argument: `ms.query.query(domain, property, relation, comparison_value)`). All that a MuSpAn query object does is store these aspects, and check that they are things MuSpAn can understand in relation to a given domain. The important thing to remember is that for each object in the domain, a MuSpAn query asks a question about how some property of that object relates to the comparison_value that can be evaluated as `True` or `False`.

The simplest way to understand this is through some examples below, but the section below contains more details about how a MuSpAn query works that can be skipped if desired. 

<details>
    
Let's start with the ```property``` we're interested in. As far as the ```query``` is concerned, a property is a tuple that specifies some value for each object. Here are a few properties of MuSpAn objects that we might want to consider:
- ('label','Celltype') -> this specifies the value of a label associated with an object, with the label name "Celltype".
- ('object','type') -> this specifies the type of an object - the value returned will be either point, line, or shape.
- ('collection',) -> this returns the set of collections that an object belongs to.

There are more possible tuples that can be understood by query as properties, including properties which specify the distance of objects from one another, or whether an object is enclosed by a shape. A full list can be found in the documentation.

A property generates an array of values, one corresponding to each object in the domain. The other parts of the query - ```relation``` and ```comparison_value``` - determine how to test each value in turn to determine whether each object matches the query. ```relation``` can be any [python operator](https://docs.python.org/3/library/operator.html), as well as a wide array of comparative statements (see documentation for full list of options), while ```comparison_value``` simply determines the value against which to test each element of the array generated by property.

To summarise: `property` is interpreted as an array of values, one for each object. Each value is in turn tested against a `comparison_value`, to see whether it matches the `relation`: the result is either `True` or `False`, for every object in the domain.
</details>



As we noted in the previous tutorial, the tuple ```('Celltype', 'Regulatory T Cell')``` that we're passing as an argument to ```visualise``` is interpreted by MuSpAn as a ```query``` object. In fact, behind the scenes, MuSpAn is converting this tuple into the query ```ms.query.query(domain, ('label','Celltype'),'is','Regulatory T Cell')```. The order of arguments in query is designed to read almost as an english language query - we want everything in this domain where the label Cluster ID is Cluster 2. Let's explicitly pass this query to visualise:

In [ ]:
# Create a query to select objects where the label 'Cluster ID' is 'Cluster 2'
query_reg_t = ms.query.query(my_domain, ('label', 'Celltype'), 'is', 'Regulatory T Cell')    

# Create a new figure for plotting
plt.figure(figsize=(10, 6))

# Visualise the domain with objects matching the query
ms.visualise.visualise(my_domain,color_by='Celltype', ax=plt.gca(), objects_to_plot=query_reg_t)

It's worth mentioning briefly the logic behind allowing certain tuples, like ```('Celltype', 'Regulatory T Cell')```, to be interpreted as queries. MuSpAn aims to be as user friendly as possible, and for a user not too familiar with coding a statement like ```ms.query.query(domain, ('label','Celltype'),'is','Regulatory T')``` is a bit of a daunting barrier. We don't want people to have to become intimately familiar with the query structure before they can plot a subset of cells, which should be a fairly straightforward task! As a result, any part of MuSpAn which accepts a query can also accept a limited number of other objects, which we call <i>query-like</i> objects.

A query-like object is anything that MuSpAn can ultimately interpret as a list of Object IDs. For completeness, the query-like things that MuSpAn recognises are:

- Queries
- Query containers - these massively boost the power of queries to ask complex questions, and will be explained in the next tutorial
- Tuples of the following forms:
  - `(label_name, label_value)` - interpreted as ```ms.query.query(domain, ('label',label_name),'is',label_value)```
  - `('collection', collection_name)` - interpreted as ```ms.query.query(domain, ('collection',),'is',collection_name)```
  - `('object', object_type)` - interpreted as ```ms.query.query(domain, ('object','type'),'is',object_type)```
  - `('label', label_name)` - interpreted as ```ms.query.query(domain, ('label',label_name),'is',True)```, i.e., the object has this label

Additionally, lists of integers are not query-like, but will be interpreted directly as a list of object IDs in most places that accept queries.

In fact, MuSpAn can handle query for asking about distances to object and whether objects are contained in another! Here is a full list of the types of queries:

- ‘label_name’ - expanded to (‘label’,’label_name’)

- (‘label’,’label_name’) - get object labels for the label ‘label_name’

- (‘object’,’type’) - get the type of each object (‘point’,’line’, or ‘shape’)

- (‘distance’, (‘centroid’, objects_1, objects_2)) - get the distance between the centroids of each objects_1 and the nearest object in objects_2 (these can be object IDs, or query-like)

- (‘distance’, (‘boundary’, objects_1, objects_2)) - get the distance between the boundaries of each objects_1 and the nearest object in objects_2 (if an object is a point, the boundary is taken to be the coordinates of the point)

- (‘collection’,) - get the collections associated with each object. Note the comma, which is required (no second argument is required here, but we need MuSpAn to recognise this as a tuple)

- (‘contains’, (contained_objects, containing_objects)) - get a boolean (True or False) for each contained_object, which is true if it is inside an element of containing_objects.

### Combining queries

Say we want to string together queries, we can use our `query_container` methods. For example, say we want all cells with CD4 intensity>15 or are labelled Epithelium then we can construct the two queries A and B.

The Boolean statements we're going to need here are:
- A: The Celltype is Epithelium
- B: The CD4 intensity value is greater than 15

...and in order to get all these objects together, we want to ask for a query that gets all objects meeting the criteria A OR B. Statements A and B can be written using the query logic that we've already seen - in fact, the relevant queries are:

In [31]:
query_A = ms.query.query(my_domain, ('label', 'Celltype'), 'is', 'Epithelium')

query_B = ms.query.query(my_domain, ('label', 'CD4'), '>', 15)

In order to ask our more complex query, we're going to need to combine these queries together. MuSpAn provides a tool for this: the <i>query container</i>. A query container is actually a very simple object: it takes in exactly two query-like objects, and combines them using one of the operators 'AND', 'OR', or 'AND NOT'. Let's make a ```ms.query.query_container()``` that asks for A OR B.

In [32]:
query_A_and_B = ms.query.query_container(query_A, 'OR', query_B)

Let's visualise to make sure we've queries the data correctly:

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(15,10))
ms.visualise.visualise(my_domain, 'Celltype', objects_to_plot=query_A, ax=ax[0,0],marker_size=5)
ax[0,0].set_title('Query A: Celltype')

ms.visualise.visualise(my_domain, 'CD4', objects_to_plot=query_B, ax=ax[0,1],marker_size=5)
ax[0,0].set_title('Query B: CD4')

ms.visualise.visualise(my_domain, 'Celltype', objects_to_plot=query_A_and_B, ax=ax[1,0],marker_size=5)
ax[1,0].set_title('Query A AND B: Celltype')

ms.visualise.visualise(my_domain, 'CD4', objects_to_plot=query_A_and_B, ax=ax[1,1],marker_size=5)
ax[1,0].set_title('Query A AND B: CD4')


Even query contrainers can be combined in query containers to make even more focused queries. Remember, we can always print our query to check what it's going to return.

In [ ]:
print(query_A_and_B)



Queries aren't just useful for visualisation, they're also a fundamental part of conducting spatial analysis. We can keep all of our data from one image within a single MuSpAn domain, and conduct different analyses on a subset of points. This is particularly useful for analysing multiplex imaging data, when we may have dozens of different cell types present, but only want to conduct a pairwise analysis between two cell types of interest. MuSpAn's query structure means that we only need to write a query identifying a population of interest once, and this query can then be passed into visualisation and analysis tools 'on the fly' to allow a targeted spatial analysis (see later tutorials).

---

### Free play: Extracting interesting subsets

Load in the `Xenium-Healthy-Colon` dataset (https://docs.muspan.co.uk/latest/muspan.datasets.html#datasets). With the help of our online tutorials, use the query infrastructure to try and extract the following conditions:

1. Let's suppose we've identified a particular cell of interest (e.g., by exploring the data in Xenium explorer), with the ID 'dfbfdjho-1', and we want to plot the location of all transcripts in the cell.

2. Suppose we now only want 'Sox9' transcripts in the cell defined by the ID 'dfbfdjho-1'.

3. Get all cells within 20µm distance of the cell with ID 'dfbfdjho-1'. Add these cells to a collection call 'Close to dfbfdjho-1'

4. Get all Transcripts within the collection 'Close to dfbfdjho-1'.


---

### Controlling the visualisation of our domains and analysis

The defaults for visualising both your spatial data (domains and objects) and output analysis are very generic and therefore you will want to adapt these for generating publication ready figures. MuSpAn allows for full user control of the visualisation of any outputs produced by the package. 

**NOTE** Any statistic computed using MuSpAn provides the numeric outputs of that method and therefore these can be plotted however you like! For example, you could pass these for Seaborn (for pretty plots in python), save the outputs to csv to plot in GraphPrism or for plotting in R's ggplot. You should have total control on how you want to present your quantative measures directly from the outputs of any MuSpAn functionality

All visualisation within MuSpAn uses ``matplotlib`` to control plot outputs. ``matplotlib`` is the standard plotting library in Python and can be used for plotting anything from points, shapes, meshes, heatmaps and more. Therefore to adapt any visualisation output within MuSpAn, we can exploit the plotting functionality of ``matplotlib`` so we suggest checking out their documentation: https://matplotlib.org/3.5.3/index.html


### Chaning the appreance of domains 

We'll start by showing how to control the visualisation of the domains using the ``ms.visualise.visualise(...)`` function. In terms of objects in our domain, we can control their appearance through the scatter and shape kwargs parameters. That is, we use

- scatter_kwargs (dict) : to control the appearance of point-like objects - checkout https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.scatter.html for all options
- shape_kwargs (dict) : to control the appearance of shape-like objects  - checkout https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Patch.html for all options

For example, if we want to have every point in our domain have a black outline to make them more distinctive, we can set:

In [ ]:
# making a dictionary of scatter plot keyword arguments - see scatter in matplotlib for more options
scatter_kwargs_updated = dict(edgecolor='k', linewidth=0.5)

ms.visualise.visualise(my_domain, 'Celltype', scatter_kwargs = scatter_kwargs_updated)


--- 

### Scale bars

We can also add scale bars to our plots using the `add_scalebar` and `scalebar_kwargs` parameters. The setting `add_scalebar=True` adds a scale to the bottom left by default and remove the axis of the plot, this will be 10% of the width of the domain if not specified. To adapt the size and appearance of the scale bar, see https://matplotlib.org/stable/api/_as_gen/mpl_toolkits.axes_grid1.anchored_artists.AnchoredSizeBar.html

Just as before, we can pass a dictionary of paremeters to `scalebar_kwargs` to control this appearance.

Let's add a scale bar of 150µm to the visualisation of our domain. 

In [ ]:
# adding a scalebar with custom size and label
scalebar_kwargs_updated=dict(size=150,label='150 µm')

ms.visualise.visualise(my_domain, 'Celltype', scatter_kwargs = scatter_kwargs_updated,add_scalebar=True,scalebar_kwargs=scalebar_kwargs_updated)

---

### Boundary Appearance


We can also adapt the appearence of the boundary line using the `boundary_kwargs` parameter. Once again, this simply takes any parameter that is use in `matplotlib.pyplot.plot` to adjust appearance. For all options, see https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html#

Let's visualise our domain with it's boundary and set the boundary to a thinner dashed line

In [ ]:
# setting boundary kwargs
boundary_kwargs_updated=dict(linewidth=2,linestyle='--')

# make sure to turn on show_boundary
ms.visualise.visualise(my_domain, 'Celltype', scatter_kwargs = scatter_kwargs_updated,add_scalebar=True,scalebar_kwargs=scalebar_kwargs_updated,show_boundary=True,boundary_kwargs=boundary_kwargs_updated)

---

### Free play: Adapting domain visualisation

Load in the `Xenium-Healthy-Colon` dataset (https://docs.muspan.co.uk/latest/muspan.datasets.html#datasets). Using the `ms.visualise.visualise` function, plot the following:

- Cell boundaries colored by cell type and shape alpha set to 1
- Add a scale bar with 200 µm
- Plot the boundary with a red dashed line
- plot Sox9 transcripts with edgecolor black

These changes the parameters of these options to make the plot as clear as possible. Is this the right boundary for the data?